In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
i = 1
authors_list = []
quotes_list = []
while True:
    
    main_url = 'https://quotes.toscrape.com'
    page_url = main_url +'/page/' + str(i)
    

    html_doc = requests.get(page_url)
    if html_doc.status_code == 200:    
        soup = BeautifulSoup(html_doc.text, 'html.parser')
        author_links = soup.select('div.quote >span > a')        
        
        for link in author_links:
            author_url = link.get('href')
            author_page_url = main_url + author_url
            html_author_page = requests.get(author_page_url)
            if html_author_page.status_code == 200:
                soup_author_page = BeautifulSoup(html_author_page.text, 'html.parser')
                fullname = soup_author_page.select('div.author-details > h3.author-title')[0].text
                born_date = soup_author_page.select('p > span.author-born-date')[0].text
                born_location = soup_author_page.select('p > span.author-born-location')[0].text
                description = soup_author_page.select('div.author-description')[0].text.strip()
                dct = {
                    "fullname": fullname,
                    "born_date": born_date,
                    "born_location": born_location,
                    "description": description
                    }
                if dct not in authors_list:
                    authors_list.append(dct)               
        quotes_divs = soup.select('div.quote')
        for quote_div in quotes_divs:
            tags = quote_div.select_one('div.tags > meta.keywords').get('content').split(',')
            author_name = quote_div.select_one('div.quote > span > small.author').text
            quote = quote_div.select_one('div.quote > span.text').text.strip()
                                
            quotes_list.append({
                "tags": tags,
                "author": author_name,
                "quote": quote
                })  
            
    next_page = soup.select('ul.pager > li.next > a')
    if not next_page:            
        break
    i+= 1

    


In [3]:
import json
with open('authors.json', 'w', encoding= 'utf-8') as f:
    json.dump(authors_list, f, ensure_ascii=False, indent=4)

In [4]:
with open('quotes.json', 'w', encoding= 'utf-8') as f2:
    json.dump(quotes_list, f2, ensure_ascii=False, indent=4)